In [2]:
from src import Match, Icsr
from src.utils import get_matches

from datetime import datetime
import random
import datasets
import pandas as pd


In [3]:
# load matches
dataset = datasets.load_dataset("FAERS-PubMed/raw_dataset")
matches = get_matches(dataset['train'])
print(len(matches))

2023-04-18 19:09:25,814 WARNING Using custom data configuration FAERS-PubMed--raw_dataset-0b83cc0b498dbbb2
2023-04-18 19:09:25,820 WARNING Found cached dataset json (/Users/kldooste/.cache/huggingface/datasets/FAERS-PubMed___json/FAERS-PubMed--raw_dataset-0b83cc0b498dbbb2/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 24.48it/s]


65648


In [5]:
# arguments
report_cutoff = 10
fulltext_only = True
commercial_only = False
test_cutoff = datetime(year=2021, month=1, day=1)

### Filter too many reports
A few articles have many reports. They are typically survey articles, drop them.

In [6]:
matches = [m for m in matches if len(m.reports) <= report_cutoff]
print(f'Matches with <= {report_cutoff} reports: {len(matches):,}')

Matches with <= 10 reports: 62,168


### Get articles with a full text

In [7]:
if fulltext_only:
    matches = [m for m in matches if m.article.fulltext]
    print(f'Matches with full text: {len(matches):,}')

Matches with full text: 18,678


## (optional) Get articles with commercial license

In [8]:
# noncommercial_licenses = {'CC BY-NC', 'CC BY-NC-SA', 'CC BY-NC-ND'}
commercial_licenses = {'CC0', 'CC BY', 'CC BY-SA', 'CC BY-ND'}

if commercial_only:
    matches = [m for m in matches if m.article.fulltext_license in commercial_licenses]
    print(f'Fulltext commercial dataset: {len(matches):,}')

## Split the data

In [9]:
def split_data(matches, test_cutoff):
    test_matches = []
    train_matches = []

    for m in matches:
        pubdate = datetime.strptime(m.article.pubdate[:4], '%Y')
        if  pubdate >= test_cutoff:
            test_matches.append(m)
        else:
            train_matches.append(m)

    print(f'Train size: {len(train_matches):,}')
    print(f'Test size: {len(test_matches):,}')

    return train_matches, test_matches

In [10]:
cutoff = datetime(year=2021, month=1, day=1)
train, test = split_data(matches, cutoff)

Train size: 14,429
Test size: 4,249


### Format the data
For every article, sample one report as target.

Drop some fields and upload to huggingface.

In [11]:

def get_icsrs_from_split(split):
    random.seed(42)

    icsrs = []
    for m in split:
        # get all valid icsrs 
        new_icsrs = [Icsr.from_report(r) for r in m.reports]
        new_icsrs = [t for t in new_icsrs if t]
        # sample one
        if new_icsrs:
            new_icsr = random.choice(new_icsrs)
            icsrs.append(new_icsr)
        else:
            icsrs.append(None)
    return icsrs

def get_ds_from_split(split):
    icsrs = get_icsrs_from_split(split)
    # only keep data with valid icsrs
    valid = [m for index,m in enumerate(split) if icsrs[index]]
    icsrs = [i for i in icsrs if i]
    print(f'Found samples with valid icsrs: {len(valid):,}')

    # format the data
    data = [m.article.dict() for m in valid]
    for d, i in zip(data, icsrs):
        d.update({
            'target': i.to_string()
        })

    df = pd.DataFrame(data=data)
    # reorder some of the columns
    df = df[['title', 'abstract','fulltext','target', 'pmid', 'fulltext_license', 'title_normalized','issue', 'pages', 'journal', 'authors', 'pubdate', 'doi', 'affiliations', 'medline_ta', 'nlm_unique_id', 'issn_linking', 'country', 'mesh_terms', 'publication_types', 'chemical_list', 'keywords', 'references', 'delete', 'pmc', 'other_id']]
    df = df.fillna('')
    ds = datasets.Dataset.from_pandas(df)
    return ds

In [12]:
train_ds = get_ds_from_split(train)
test_ds = get_ds_from_split(test)

train_ds = train_ds.shuffle(42)
test_ds = test_ds.shuffle(42)

Found samples with valid icsrs: 12,031
Found samples with valid icsrs: 3,628


In [17]:
# create a validation set
len_train = int(len(train_ds) * 0.8)

val_ds = train_ds.select(range(len_train,len(train_ds)))
train_ds = train_ds.select(range(len_train))

In [18]:
print('train size: ', len(train_ds))
print('val size: ', len(val_ds))
print('test size: ', len(test_ds))

train size:  9624
val size:  2407
test size:  3628


### Upload the data

In [19]:
ds = datasets.DatasetDict({
    'train': train_ds,
    'val': val_ds,
    'test': test_ds
})

ds.push_to_hub('FAERS-PubMed/BioDEX-ICSR', private=True)

2023-04-18 19:16:11,114 WARNING Pushing split train to the Hub.
Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  1.98it/s]
2023-04-18 19:18:07,415 WARNING Pushing split val to the Hub.
Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  2.90it/s]
2023-04-18 19:18:37,198 WARNING Pushing split test to the Hub.
Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]